In [27]:
import os
import subprocess
from astropy.table import Table
import numpy as np
from desitarget.targetmask import desi_mask, bgs_mask, mws_mask, obsmask, obsconditions
import fitsio

os.environ['DECALS_PATH'] = '/global/project/projectdirs/cosmo/data/legacysurvey/dr6/'
minitestdir = os.getcwd()
minitestdata = minitestdir.replace('src', 'data')
targetfile = os.path.join(minitestdata, 'dr6_targets.fits')
print(minitestdir)

/global/cscratch1/sd/forero/decals_fiberaassign_hack/src


In [2]:
cmd = "select_targets {source} {destination}"
cmd = cmd.format(source=os.environ['DECALS_PATH'], destination=targetfile)
print(cmd)

select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr6/ /global/cscratch1/sd/forero/decals_fiberaassign_hack/data/dr6_targets.fits


In [27]:
subprocess.call(cmd.split())

0

In [28]:
columns = [
    'TARGETID', 'RA', 'DEC', 'SUBPRIORITY', 'BRICKNAME',
    'DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET',
]
targetdata = fitsio.read(targetfile, 'TARGETS', columns=columns)

In [31]:
targetdata[:10]

array([ (b'1475p292',  147.57113435,  29.23259145,  0.43693441, 26390344471937029, 1152921504606846976, 131074, 0),
       (b'1475p292',  147.47009862,  29.23390583,  0.41970843, 26390344471937031, 1152921504606846976, 131074, 0),
       (b'1475p292',  147.44487668,  29.23485253,  0.34180474, 26390344471937032, 1152921504606846976, 131074, 0),
       (b'1475p292',  147.44442566,  29.23472314,  0.98543102, 26390344471937034, 1152921504606846976, 131074, 0),
       (b'1475p292',  147.50860877,  29.23243401,  0.46081674, 26390344471937039, 1152921504606846976, 131074, 0),
       (b'1475p292',  147.48878798,  29.23705999,  0.63215601, 26390344471937071, 1152921504606846976, 131074, 0),
       (b'1475p292',  147.64479256,  29.23732863,  0.41369352, 26390344471937081, 1152921504606846976,  65537, 0),
       (b'1475p292',  147.4307453 ,  29.23938314,  0.34730127, 26390344471937104, 1152921504606846976, 131074, 0),
       (b'1475p292',  147.44969646,  29.24107448,  0.02729205, 2639034447193712

In [32]:
print('MWS_TARGETS: {}'.format(np.count_nonzero(targetdata['MWS_TARGET']!=0)))
print('BGS_TARGETS: {}'.format(np.count_nonzero(targetdata['BGS_TARGET']!=0)))
print('DESI_TARGETS: {}'.format(np.count_nonzero(targetdata['DESI_TARGET']!=0)))

MWS_TARGETS: 0
BGS_TARGETS: 13139177
DESI_TARGETS: 25887442


In [38]:
#(Here I use sky.py to generate the sky positions file)

In [33]:
print('ELG {}'.format(np.count_nonzero((targetdata['DESI_TARGET']&desi_mask.ELG)!=0)))
print('QSO {}'.format(np.count_nonzero((targetdata['DESI_TARGET']&desi_mask.QSO)!=0)))
print('LRG {}'.format(np.count_nonzero((targetdata['DESI_TARGET']&desi_mask.LRG)!=0)))

ELG 9806142
QSO 1145240
LRG 2137370


In [34]:
print('BGS_FAINT {}'.format(np.count_nonzero((targetdata['BGS_TARGET']&bgs_mask.BGS_FAINT)!=0)))
print('BGS_BRIGHT {}'.format(np.count_nonzero((targetdata['BGS_TARGET']&bgs_mask.BGS_BRIGHT)!=0)))


BGS_FAINT 3341902
BGS_BRIGHT 9797275


In [35]:
darkstd = (targetdata['DESI_TARGET'] & desi_mask.mask('STD_FSTAR|STD_WD')) != 0
brightstd = (targetdata['DESI_TARGET'] & desi_mask.mask('STD_BRIGHT')) != 0
print('{} dark standards'.format(np.count_nonzero(darkstd)))
print('{} bright standards'.format(np.count_nonzero(brightstd)))

83031 dark standards
18450 bright standards


In [11]:
import desitarget.mtl
mtl = desitarget.mtl.make_mtl(targetdata)
mtl.meta['EXTNAME'] = 'MTL'
mtl.write('../data/mtl.fits')

INFO:mtl.py:82:make_mtl: 88724 of 25887442 targets have priority zero, setting N_obs=0.


In [36]:
darkstd = (targetdata['DESI_TARGET'] & desi_mask.mask('STD_FSTAR|STD_WD')) != 0
brightstd = (targetdata['DESI_TARGET'] & desi_mask.mask('STD_BRIGHT')) != 0

In [37]:
print(np.count_nonzero(darkstd))
print(np.count_nonzero(brightstd))

83031
18450


In [38]:
darkdata = targetdata[darkstd]
obscond = np.zeros(len(darkdata), dtype=np.int64)
darkdata = np.lib.recfunctions.append_fields(
    darkdata, 'OBSCONDITIONS', obscond)
fitsio.write('../data/std-dark.fits', darkdata, extname='STD')

In [ ]:
brightdata = targetdata[brightstd]
obscond = np.zeros(len(brightdata), dtype=np.int64)
brightdata = np.lib.recfunctions.append_fields(
    brightdata, 'OBSCONDITIONS', obscond)
fitsio.write('../data/-dark.fits', brightdata, extname='STD')

In [21]:
## Add an OBSCONDITIONS column
# This is required by the fiberassign I/O, even for standards and skies,
# even though it isn't currently used for those.

obscond = np.zeros(len(targetdata), dtype=np.int64)
obscond[darkstd] |= obsconditions.mask('DARK|GRAY')
obscond[brightstd] |= obsconditions.mask('BRIGHT')

targetdata = np.lib.recfunctions.append_fields(
    targetdata, 'OBSCONDITIONS', obscond)

## Output standard files

fitsio.write('../data/std-dark.fits', targetdata[darkstd], extname='STD')
fitsio.write('../data/std-bright.fits', targetdata[brightstd], extname='STD')

KeyboardInterrupt: 

In [19]:
obsconditions.mask

<bound method BitMask.mask of obsconditions:
  - [DARK,             0, "Moon is down"]
  - [GRAY,             1, "Moon up; illum fraction<0.6 and (illum*elev)<30 deg"]
  - [BRIGHT,           2, "Moon up and bright (not GRAY)"]
  - [POOR,             3, "Very bad seeing, high extinction, or bright cloud cover"]
  - [TWILIGHT12,       4, "Between sunset/sunrise and 12 degree twilight"]
  - [TWILIGHT18,       5, "Between 12 and 18 degree twilight"]
  - [DAY,              6, "Daytime calibrations"]
  - [CLOSED,           7, "Nighttime but dome is closed due to rain, wind, dew..."]
  - [APOCALYPSE,       8, "Conditions are so bad that the world is ending anyway"]>